In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [4]:
all_data = pd.read_csv("acled_data.csv")


In [27]:
print(all_data.head()["notes"][0])
all_data.head()

On 23 August 2024, Turkish army warplanes caried out airstrikes targeting 5 unspecified villages in Jabal Matin region (Al Amadiya, Duhok), causing significant fires. Casualties unknown.


,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,IRQ58256,23 August 2024,2024,1,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Turkey (2016-),NaN,8,...,Jabal Matin,37.1071,43.5172,2,Baghdad Today; ROJ News,National-Regional,"On 23 August 2024, Turkish army warplanes cari...",0,NaN,1724706546
1,IRQ58259,23 August 2024,2024,1,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Turkey (2016-),NaN,8,...,Gire Bahar,37.0992,43.6066,2,ANF News,International,"On 23 August 2024, Turkish army warplanes carr...",0,NaN,1724706546
2,MEX91579,23 August 2024,2024,1,Strategic developments,Strategic developments,Looting/property destruction,Military Forces of Mexico (2018-),NaN,1,...,Coast of Colima,18.9474,-104.3941,3,MiMorelia,National,"Property destruction: On 23 August 2024, off t...",0,NaN,1724706548
3,MEX91580,23 August 2024,2024,1,Strategic developments,Strategic developments,Looting/property destruction,Military Forces of Mexico (2018-),NaN,1,...,Coast of Michoacan,18.0052,-103.3520,2,MiMorelia,Subnational,"Property destruction: On 23 August 2024, off t...",0,NaN,1724706548
4,LBN23478,23 August 2024,2024,1,Political violence,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Israel (2022-),NaN,8,...,Tallit el Azziye,33.2930,35.5419,1,Liveuamap; National News Agency Lebanon; Telegram,Local partner-New media,"On 23 August 2024, the Israeli military fired ...",0,NaN,1724706549


In [72]:
for column in all_data:
    print(column)

all_data['country'].unique()
relevant_columns = all_data[["event_id_cnty","event_date","event_type","actor1","actor2","interaction", "region", "country","location","latitude","longitude","fatalities","notes", "source"]]

event_id_cnty
event_date
year
time_precision
disorder_type
event_type
sub_event_type
actor1
assoc_actor_1
inter1
actor2
assoc_actor_2
inter2
interaction
civilian_targeting
iso
region
country
admin1
admin2
admin3
location
latitude
longitude
geo_precision
source
source_scale
notes
fatalities
tags
timestamp


In [6]:
scs_countries = ["China", "Philippines", "Vietnam", "Taiwan","Brunei", "Indonesia", "Malaysia"]

scs_data = relevant_columns[relevant_columns['country'].isin(scs_countries)]


scs_data[scs_data['event_type'].isin(['Violence against civilians',
       'Strategic developments', 'Battles', 'Explosions/Remote violence'])]

scs_data['event_type'].unique()

mask = (scs_data["actor1"].str.contains('china', case=False, na=False) | (scs_data["actor2"].str.contains('china', case=False, na=False)))

filtered_data = scs_data[mask & scs_data['event_type'].isin(['Violence against civilians',
       'Strategic developments', 'Battles', 'Explosions/Remote violence'])]

print(filtered_data["notes"])


4237      Other: On 19 August 2024, China Coast Guard (C...
5934      On 17 August 2024, a few hundred police arrest...
7012      Around 16 August 2024 (as reported), a Qinhuan...
7510      Around 16 August 2024 (as reported), it is sus...
14305     On 8 August 2024, a Sichuan-based female petit...
                                ...                        
342515    On 1 September 2023, dozens of government offi...
342602    On 1 September 2023, two people claying to be ...
342719    On 1 September 2023, unidentified people detai...
345023    On 29 August 2023, the Indonesian police force...
345194    Other: On 29 August 2023, the Societas Linguis...
Name: notes, Length: 200, dtype: object


In [59]:
ccg_t_mask = (scs_data["notes"].str.contains('CCG', case=False, na=False) | scs_data["notes"].str.contains("Taiwan", case=False, na=False))

ccg_t_data = filtered_data[ccg_t_mask]

ccg_t_data.loc[:,("event_date")] = pd.to_datetime(ccg_t_data.loc[:,"event_date"], format='%d %B %Y').dt.date

ccg_t_data

/var/folders/1k/3mrcqp5d4670v77qsx12xwmm0000gn/T/ipykernel_48604/1522458937.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ccg_t_data = filtered_data[ccg_t_mask]


,event_id_cnty,event_date,event_type,actor1,actor2,interaction,region,country,location,latitude,longitude,fatalities,notes
4237,PHL17114,2024-08-19,Strategic developments,Military Forces of China (2012-),Military Forces of the Philippines (2022-) Coa...,18,Southeast Asia,Philippines,Escoda Shoal,9.7456,116.5226,0,"Other: On 19 August 2024, China Coast Guard (C..."
62239,PHL16870,2024-06-17,Strategic developments,Military Forces of China (2012-),Military Forces of the Philippines (2022-) Phi...,18,Southeast Asia,Philippines,Ayungin Shoal,9.7188,115.8888,0,"Other: On 17 June 2024, at least 8 Philippine ..."
89840,PHL16871,2024-05-19,Strategic developments,Military Forces of China (2012-),Military Forces of the Philippines (2022-) Phi...,18,Southeast Asia,Philippines,Ayungin Shoal,9.7188,115.8888,0,"Other: On 19 May 2024, China Coast Guard (CCG)..."
109349,PHL16756,2024-04-30,Strategic developments,Military Forces of China (2012-),Government of the Philippines (2022-),18,Southeast Asia,Philippines,Panatag Shoal,15.2180,117.7291,0,"Other: On 30 April 2024, China Coast Guard (CC..."
142794,PHL16645,2024-03-23,Strategic developments,Military Forces of China (2012-),Military Forces of the Philippines (2022-) Coa...,18,Southeast Asia,Philippines,Ayungin Shoal,9.7188,115.8888,0,"Other: On 23 March 2024, China Coast Guard (CC..."
161030,PHL16589,2024-03-05,Strategic developments,Military Forces of China (2012-),Military Forces of the Philippines (2022-) Coa...,18,Southeast Asia,Philippines,Ayungin Shoal,9.7188,115.8888,0,"Other: On 5 March 2024, China Coast Guard (CCG..."
244289,PHL16329,2023-12-10,Strategic developments,Military Forces of China (2012-),Military Forces of the Philippines (2022-) Coa...,18,Southeast Asia,Philippines,Ayungin Shoal,9.7188,115.8888,0,"Other: On 10 December 2023, China Coast Guard ..."
245452,PHL16326,2023-12-09,Strategic developments,Military Forces of China (2012-),Government of the Philippines (2022-),18,Southeast Asia,Philippines,Panatag Shoal,15.2180,117.7291,0,"Other: On 9 December 2023, China Coast Guard (..."
272591,PHL16180,2023-11-10,Strategic developments,Military Forces of China (2012-),Military Forces of the Philippines (2022-) Coa...,18,Southeast Asia,Philippines,Ayungin Shoal,9.7188,115.8888,0,"Other: On 10 November 2023, China Coast Guard ..."
283060,CHN13872,2023-10-31,Strategic developments,Military Forces of China (2012-),NaN,10,East Asia,China,Taiwan Strait,25.7699,119.7183,0,"Other: On 31 October 2023, the People's Libera..."


In [110]:
yemen_mask = (all_data["notes"].str.contains("Houthi", case=False, na=False) & (all_data["notes"].str.contains("IMO", case=True, na=False)))

yemen = relevant_columns[yemen_mask & relevant_columns['event_type'].isin(['Violence against civilians',
       'Strategic developments', 'Battles', 'Explosions/Remote violence'])]

yemen

,event_id_cnty,event_date,event_type,actor1,actor2,interaction,region,country,location,latitude,longitude,fatalities,notes,source
915,YEM91147,22 August 2024,Explosions/Remote violence,Military Forces of Yemen (2017-) Houthi,NaN,10,Middle East,Yemen,Gulf of Aden,12.5340,47.8580,0,"On 22 August 2024, between 12:00 am and 1:00 a...",Reuters; UKMTO; Yemeni Army - Houthi
916,YEM91148,22 August 2024,Explosions/Remote violence,Military Forces of Yemen (2017-) Houthi,Civilians (International),17,Middle East,Yemen,Red Sea,21.2712,38.2520,0,"On 22 August 2024, around 8:00 pm Sanaa time, ...",Reuters; UKMTO; Yemeni Army - Houthi
1847,YEM91098,21 August 2024,Battles,Military Forces of Yemen (2017-) Houthi,Private Security Forces (International),18,Middle East,Yemen,South Red Sea,16.1382,41.4067,0,"On 21 August 2024, At around 6:15 am Sanaa tim...",UKMTO; Yemeni Army - Houthi
1848,YEM91099,21 August 2024,Explosions/Remote violence,Military Forces of Yemen (2017-) Houthi,Civilians (International),17,Middle East,Yemen,South Red Sea,16.1382,41.4067,0,"On 21 August 2024, between 7am and 9am Sanaa t...",Al Masdar; Al Mashhad Al Yemeni - Pro-IRG; EUN...
1849,YEM91140,21 August 2024,Explosions/Remote violence,Military Forces of Yemen (2017-) Houthi,NaN,10,Middle East,Yemen,Gulf of Aden,12.5340,47.8580,0,"On 21 August 2024, around 11:45 am Sanaa time,...",Reuters; UKMTO; Yemeni Army - Houthi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251088,YEM83984,03 December 2023,Explosions/Remote violence,Military Forces of Yemen (2017-) Houthi,Civilians (International),17,Middle East,Yemen,Strait of Bab el Mandeb,13.2694,42.9758,0,"On 3 December 2023, at around 4:30 pm Sanaa ti...",AP; Barrons; USCENTCOM; Yemen Data Project; Ye...
256432,YEM84665,27 November 2023,Explosions/Remote violence,Military Forces of Yemen (2017-) Houthi,NaN,10,Middle East,Yemen,South Red Sea,16.1382,41.4067,0,"On 27 November 2023, unidentified forces (like...",MSCHOA
256433,YEM83954,27 November 2023,Explosions/Remote violence,Military Forces of Yemen (2017-) Houthi,NaN,10,Middle East,Yemen,Gulf of Aden,12.5340,47.8580,0,"On 27 November 2023, suspected Houthi forces l...",Aden al Ghad; Al Masdar; Navy Times; The Washi...
257102,YEM83957,26 November 2023,Strategic developments,Unidentified Armed Group (Somalia),Military Forces of the United States (2021-),38,Middle East,Yemen,Gulf of Aden,12.5340,47.8580,0,"Other: On 26 November 2023, Somali gunmen boar...",Aden al Ghad; Jewish Press; Reuters; The Washi...


In [111]:
for id, event_id, note,  in yemen.loc[:, ["event_id_cnty", "notes"]].itertuples():
    print(id, event_id, note)

915 YEM91147 On 22 August 2024, between 12:00 am and 1:00 am Sanaa time, Houthi forces launched 2 drones/missiles (coded as missiles based on other activity in the Red Sea) targeting the SW North Wind I bulk carrier (IMO: 9514004) in the Gulf of Aden. The Houthis claimed the company owning the ship was in violation of the Houthi ban to call at Israeli ports. The ship reported 2 explosions in close proximity to the ship. There were no fatalities or injuries, and the ship continued its route. The ship was targeted several times between 21 and 22 August (coded as separate events).
916 YEM91148 On 22 August 2024, around 8:00 pm Sanaa time, Houthi forces launched 1 drone boat targeting the SW North Wind I bulk carrier (IMO: 9514004) in the South Red Sea around 55 nm west of Hudaydah. The Houthis claimed the company owning the ship was in violation of the Houthi ban to call at Israeli ports. The ship reported 1 explosion in close proximity to the ship that caused minor damages. There were no

In [123]:
imo_dict = {}

for _, row in yemen.iterrows():
    note = row['notes']
    
    imo_match = re.search(r'IMO\s*:\s*(\d+)', note)
    if imo_match:
        imo_number = imo_match.group(0)
        if imo_number in imo_dict:
            imo_dict[imo_number]['notes'].append(note)
        else:
            imo_dict[imo_number] = {'notes': [note]}

'''for imo_number, entry in imo_dict.items():
    if len(entry['notes']) > 1:
        print(imo_number, entry)'''

SyntaxError: incomplete input (2816386933.py, line 16)